In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")

In [8]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain

In [15]:
db = SQLDatabase.from_uri("postgresql://postgres:@localhost:5432/postgres?options=-csearch_path%3Dbooks_recommendation")


/opt/anaconda3/lib/python3.12/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Did not recognize type 'geography' of column 'location'
  self._metadata.reflect(


In [21]:
db.get_usable_table_names()

['books',
 'city',
 'location',
 'spatial_ref_sys',
 'status',
 'user_books',
 'users']

In [18]:
chain = create_sql_query_chain(llm = llm, db = db)

In [37]:
database_description = (
    "The database contains a table named `books_recommendation.books`, which stores detailed information about books. "
    "This is a PostgreSQL database, so please ensure that all queries are compatible with PostgreSQL syntax.\n\n"
    "The `books_recommendation.books` table includes the following columns:\n"
    "- `book_hash`: A unique identifier for each book.\n"
    "- `title`: The name of the book.\n"
    "- `author`: The name of the book's author.\n"
    "- `slug`: A unique key used to generate a distinct URL for each book.\n\n"
    "SQLQuery: Please write your query below following this format."
)"

response = chain.invoke({"question": database_description + " Who is the author of the book 'The Light We Lost' ?"})

In [35]:
import re
def extract_sql_query(response):
    # Define the regular expression pattern to match the SQL query
    pattern = re.compile(r'SQLQuery:\s*(.*)')
    # Search for the pattern in the response
    match = pattern.search(response)
    if match:
        # Extract and return the matched SQL query
        return match.group(1).strip()
    else:
        return None
sql_query = extract_sql_query(response)

In [39]:
sql_query="SELECT author FROM books_recommendation.books WHERE title  = \'My Name Is Asher Lev\';"

In [40]:
result = db.run(sql_query)
print(result)

[('Chaim Potok',)]
